In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

In [3]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="async_youtuber/ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
    output_file="async_youtuber/selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="async_youtuber/research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="async_youtuber/competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="async_youtuber/inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="async_youtuber/script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)

result

 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
I need to come up with engaging and viral video ideas for a YouTube channel focused on the hot sauce industry. The target audience is likely young, adventurous, and interested in food culture, so the content should be both entertaining and informative. 

Action: Delegate work to co-worker  
Action Input: { "co-worker": "Senior Researcher", "task": "Research trending topics and themes in the food and hot sauce industry, especially those appealing to a young audience. Look for unique angles that have the potential to go viral, such as challenges, reviews, cultural aspects, or behind-the-scenes content.", "context": "The goal is to come up with 5 compelling video ideas for a YouTube channel that focuses on hot sauce. The ideas should be innovative, engaging, and designed to capture the interest of a yo

'**[Video Title: "Spice It Up! Exploring the Hot Sauce Revolution of 2023"]**\n\n---\n\n**[Intro: 0:00 - 0:15]**  \n*Visual: Colorful montage of hot sauces being poured, spicy dishes, and Gen Z enjoying hot sauce in various settings*  \n"Welcome back to our channel! Today, we’re diving into the vibrant world of hot sauce and how it’s being redefined in 2023! From Gen Z trends to artisanal creations, let’s explore what’s heating things up!"\n\n---\n\n**[Segment 1: Gen Z Hot Sauce Culture - 0:15 - 1:00]**  \n*Visual: Clips of young people at restaurants, bringing their own hot sauces, and social media posts featuring hot sauce*  \n"First up, let’s talk about Gen Z—the driving force behind a spicy renaissance! Did you know that over half of Gen Z consumers consider themselves hot sauce aficionados? Many are even bringing their own hot sauces to restaurants! This generation loves adventurous flavors, with 62% more likely to try a dish if it’s labeled as spicy. It’s a whole new era for hot 